# Infrastructure 
Cabe ao usuario definir algumas poucas configuracoes com relacao à arquitetura interna do dataquality. Questoes como por exemplo, tamanho de repositórios e persistencia ou nao de alguns registros.</br>
Como será explicado nos próximos tópicos, a definicao desse arquivo é um pouco mais complexa quando estamos configurando uma arquitetura para `anomalyDetection`

### infraYaml:
Seja `infrastructure.yaml` um arquivo já existente e costruido com a formatacao necessária.

```python
dqView_simple = {"viewName" : "foo",
                 "inputData": "foo",
                 "infraYaml": "yamlFiles/complex_tutorial/basic_anomalyDetec/infrastructure.yaml",
                 "vsYaml": "foo"}
```


### Infrastructure .yaml pattern

## AnomalyDetection
O arquivo .yaml deve seguir a seguinte formatacao:

```yaml
persistentRepository: {
    maxSize: 30
}
temporaryRepository: {
    maxSize: 10, 
    filters: {
        lastNDays: 15,
        onlySucceed: True, 
        nDaysAgo: 15, #NotImplementedYet
        
        tags: { #NotImplementedYet
            dayType: "business day" 
        },
        tagsExpression: (((DayType:"business day") or (DayType:"Exception")) and (AccessLevel:"Public")) #NotImplementedYet
    }
}
```

##### persistentRepository
Ao final de cada execucao do processo de DataQuality, as métricas calculadas e os resultados obtidos sao todos armazenado no diretório de registros persistentes. __Com o registro persistente, é possivel ter um histórico das execucoes passadas.__

| p1                   | p2       | p3   | p4   | Desricao                                                                                       | parametro obrigatório   |
|:---------------------|:---------|:-----|:-----|:-----------------------------------------------------------------------------------------------|:------------------------|
| persistentRepository |          |      |      | É no persistentRepository onde os registro históricos das ultimas execucoes serao armazendados | True                    |
| persistentRepository | maxSize  |      |      | Número máximo de registros que serao armazenado no repositório                                 | True                    |

##### temporaryRepository

É a partir da diferenca entre as métricas do dataframe atual e as métricas calculadas nas execucoes anteriores que o processo de AnomalyDetection acontece. A infraestrutura do sistema já possui um registro histórico de metricas, o persistentRepository. Contudo, para trazer mais flexibilidade ao processo, __o temporaryRepository permite uma selecao de quais dos registro históricos que serao utilizado na deteccao de anomalias.__<br>
O objetivo do temporaryRepository é criar um repositório temporario e pertinente apenas à execucao atual, contendo dados filtrados da base de dados persistentes do sistema.

| p1                  | p2       | p3               | p4      | Desricao                                                                                                                                                                                                                                                          | parametro obrigatório   |
|:--------------------|:---------|:-----------------|:--------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------|
| temporaryRepository |          |                  |         | É no temporaryRepository onde o os registros históricos filtrados seram armazenados temporariamente para suportar o processo de deteccao de anomalias                                                                                                             | True                    |
| temporaryRepository | maxSize  |                  |         | Número máximo de registros que serao armazenado no repositório                                                                                                                                                                                                    | True                    |
| temporaryRepository | filters  |                  |         | Definicao dos filtros que serao aplicados ao persistentRepository, de modo a criar o temporaryRepository                                                                                                                                                          | False                   |
| temporaryRepository | filters  | lastNDays        |         | Filtrar por registro históricos que estejam no maximo N dias distantes do registro mais recente (independente da data corrente, se baseia apenas nos ultimos registros que estao em histórico)                                                                    | False                   |
| temporaryRepository | filters  | onlySucceed      |         | Filtrar apenas por registros históricos que tenham sido aprovados na analise de Dq. Caso o registro faca parte de uma execucao que nao tenha sido validada, ele nao ira compor o conjunto de dados a ser utilizado na deteccao de anomalias do corrente dataframe | False                   |
| temporaryRepository | filters  | nDaysAgo         |         | Filtrar por datas históricas de no máximo N dias atrás da data atual (considera a data corrente em que o dq esta sendo executado)                                                                                                                                 | False                   |
| temporaryRepository | filters  | tags             |         | Filtrar apenas por registro que tenham uma das tags listadas neste parametro                                                                                                                                                                                      | False                   |
| temporaryRepository | filters  | tags             | tagName: value | Especificar o nome e valor da tag a ser filtrada                                                                                                                                                                                                                  | False                   |
| temporaryRepository | filters  | tagsExpression   |         | Filtrar por registro que tenham as tags, de acordo com a expressao lógica especificada neste parametro                                                                                                                                                            | False                   |